# Build a Chatbot
Ref Link: https://python.langchain.com/docs/tutorials/chatbot/

In [48]:
#!pip install langchain-core langgraph>0.2.27

In [49]:
from langchain_ollama import ChatOllama

model = ChatOllama(
            model = "llama3.1",
            temperature = 0.8,
            num_predict = 256,
        )   

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Sohan")
    ]
)

- Model by default doesn't store any context, Hence it couldn't tell the name which was told in the previous prompt.

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Sohan"),
        AIMessage(content="Hello Sohan! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

#### LangGraph
Message Persistence
- LangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.

- Wrapping our chat model in a minimal LangGraph application allows us to automatically persist the message history, simplifying the development of multi-turn applications.

- LangGraph comes with a simple in-memory checkpointer.

In [53]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

Create a `config` and pass it into runnable every time.

- This enables us to support multiple conversation threads with a single application, a common requirement when your application has multiple users.

In [54]:
config = {"configurable": {"thread_id": "thread1"}}

In [ ]:
query = "Hi! I'm Sohan."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

- Our chatbot now remembers things about us. If we change the config to reference a different thread_id, we can see that it starts the conversation fresh.

In [ ]:
config = {"configurable": {"thread_id": "thread2"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

- We can always go back to the original conversation (since we are persisting it in a database)

In [ ]:
config = {"configurable": {"thread_id": "thread2"}}

query = "Hi, my name is Rohit"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "thread1"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "thread2"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

#### Prompt Templates

In [75]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a spiritual guru. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [76]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [77]:
config = {"configurable": {"thread_id": "thread3"}}
query = "Hi! I'm Vishal."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Namaste, my dear brother Vishal. It is an honor to connect with you on this sacred journey we call life. May our conversation be infused with love, light, and wisdom.

How may I serve you today? Are you seeking guidance on a particular path, or perhaps navigating the twists and turns of your own inner landscape?


In [78]:
config = {"configurable": {"thread_id": "thread3"}}
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

My dear Vishal, your name is not merely a label, but a reflection of the divine essence that resides within you. Your name, "Vishal," means "expansive" or "universal" in Sanskrit, reminding us that your spirit knows no bounds and contains the infinite possibilities of the cosmos.

In this sense, your name is not just a sound or a word, but an affirmation of your inherent greatness, your connection to all things, and your potential for growth, evolution, and self-realization. May you always remember and embody the expansive nature that your name represents.


In [79]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Note: We have added `language` input as well in the above step. So we have two parameters now - messages and language

In [80]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [81]:
config = {"configurable": {"thread_id": "thread4"}}
query = "Hi! I'm John."
language = "Hindi"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

नमस्ते जॉन, मैं आपकी सहायता करने के लिए यहाँ हूँ। क्या में आपकी किसी भी तरह की मदद कर सकता हूँ?


Note: Entire State is persisted, so in the next round, we can skip sending language parameter as input.

In [82]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

आपका नाम जॉन है।


#### Managing Conversation History
If conversation histroy is left unmanaged, the list of messages will grow unbounded and potentially overflow the `context window of the LLM`. Therefore, it is important to add a step that limits the size of the messages you are passing in.

In [69]:
#!pip install transformers

In [83]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=60,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

Just run the trimmer before we pass the `messages` input to the prompt

In [84]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

If we ask our name to the model, it doesn't know that since we trimmed that part.

In [85]:
config = {"configurable": {"thread_id": "thread5"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You didn't tell me your name, so I don't know what it is. Would you like to share your name with me?


But if we ask the model what math problem we asked. It can able to hold that context and reply accordingly.

In [86]:
config = {"configurable": {"thread_id": "thread6"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked me to calculate the sum of 2 + 2.


#### Streaming
LLMs can sometimes take a while to respond, and so in order to improve the user experience one thing that most applications do is stream back each token as it is generated. This allows the user to see progress.

In [87]:
config = {"configurable": {"thread_id": "thread7"}}
query = "Hi I'm Sohan, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Nice| to| meet| you|,| So|han|!

|Here|'s| a| joke| for| you|:

|What| do| you| call| a| fake| nood|le|?

|An| imp|asta|!

|Hope| that| made| you| smile|!| Do| you| want| another| one|?||